<a href="https://colab.research.google.com/github/sarunsmenon/llm/blob/main/LLM_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries

In [19]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import chromadb
import random
from dotenv import load_dotenv
import pandas as pd
load_dotenv(dotenv_path="../*.env")

True

In [2]:
from deepeval.synthesizer import Synthesizer
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase

In [3]:
import requests
import chromadb
from bs4 import BeautifulSoup
import openai
import hashlib
import time
import re

# Load Variables

In [4]:
llm_model = 'gpt-4.1' # 'gpt-3.5-turbo-1106'
llm = ChatOpenAI(model=llm_model, temperature=0)

In [5]:
# Start crawling from the initial URL
ignore_lst = []
max_pg_lmt = 5000
fldr = '../data/mhdb'

# Load Contexts

## Load Chroma Collection

In [6]:
chroma_client = chromadb.PersistentClient(path=fldr)
collection = chroma_client.get_or_create_collection(name="semantic")

## Load Contexts

In [7]:
all_contexts = collection.get(include=["documents"])["documents"]
# Sample 10 contexts from the collection
sample_context = random.sample(all_contexts, 10)
sample_context_lst = [[context] for context in sample_context]
print(f"Sampled {len(sample_context_lst)} contexts for evaluation.")

Sampled 10 contexts for evaluation.


# Synthesizer

In [8]:
batch_size = 2  # Adjust as needed
all_goldens = []

for i in range(0, len(sample_context_lst), batch_size):
    batch_contexts = sample_context_lst[i:i + batch_size]
    synthesizer = Synthesizer(model=llm_model)
    synthesizer.generate_goldens_from_contexts(
        contexts=batch_contexts,
        max_goldens_per_context=2
    )
    all_goldens.extend(synthesizer.synthetic_goldens)

Output()

Output()

Output()

Output()

Output()

In [ ]:
# use this code when we want to generate goldens from contexts
# Uncomment the following lines to generate synthetic data from contexts
# 
# from deepeval.synthesizer import Synthesizer
# synthesizer = Synthesizer(model=llm_model)
# synthesizer.generate_goldens_from_contexts(
#     # Provide a list of context for synthetic data generation
#     contexts=sample_context_lst,
#     max_goldens_per_context = 2
# )

In [9]:
len(all_goldens)

70

In [10]:
test_cases = []

for golden, retrieval_ctx in zip(all_goldens, sample_context_lst):
    res = llm.invoke(golden.input).content
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=res,
        expected_output=golden.expected_output,
        retrieval_context=retrieval_ctx
    )
    test_cases.append(test_case)

# Evaluation

In [11]:
clean_test_case_lst = []
for test_case in test_cases:
  if isinstance(test_case, LLMTestCase):
    clean_test_case_lst.append(test_case)


In [12]:
print(len(test_cases))
print(len(clean_test_case_lst))

10
10


In [14]:
random_test_case_lst = random.sample(clean_test_case_lst, 10)
all_results = []
batch_size = 5 # Adjust this number to control concurrency


metrics_lst = [
    AnswerRelevancyMetric(threshold=0.7, verbose_mode=False), # Set verbose_mode to False
    FaithfulnessMetric(threshold=0.7, verbose_mode=False) # Do the same for other metrics
]

for i in range(0, len(random_test_case_lst), batch_size):
    batch = random_test_case_lst[i:i + batch_size]
    print(f"Evaluating batch {i // batch_size + 1}...")
    results = evaluate(test_cases=batch, metrics=metrics_lst)

    all_results.extend(results)

print("Evaluation complete for all batches.")

Evaluating batch 1...


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4.1, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 1.00 because the answer is fully relevant, directly addressing the question with no irrelevant statements. Great job staying focused and on-topic!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: Great job! There are no contradictions, so the actual output is fully faithful to the retrieval context., error: None)

For test case:

  - input: What evidence and practices regarding drug consumption rooms are detailed by Nougier and Schatz (2012)?
  - actual output: Certainly! Here’s a summary of the evidence and practices regarding drug consumption rooms (DCRs) as detailed by Nougier and Schatz (2012):

**Reference:**  
Nougier, M. & Schatz, E. (2012). *Drug Consumption Rooms: Evidence and Practice*. International Drug Policy Consortium (IDPC) Briefing Paper.

---

### Evidence and Practices Deta

...

Output()

ERROR:root:OpenAI Error: Could not parse response content as the length limit was reached - 
CompletionUsage(completion_tokens=32768, prompt_tokens=1269, total_tokens=34037, 
completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0,
rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)) 
Retrying: 1 time(s)...



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 1.00 because the answer was fully relevant and addressed the input directly, with no irrelevant statements present. Great job staying focused and concise!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: Great job! There are no contradictions, so the actual output is fully faithful to the retrieval context., error: None)

For test case:

  - input: Summarize international policy differences in supervised injection facility regulations as discussed by Nougier & Schatz (2012).
  - actual output: Certainly! According to Nougier & Schatz (2012), international policy differences regarding supervised injection facilities (SIFs) are significant. Some countries, such as Switzerland, Germany, the Netherlands, and Australia, have established legal frameworks that permit and regulate SIFs as part of har

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

Evaluation complete for all batches.


In [15]:
all_results[0][1][0]

TestResult(name='test_case_3', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.7, success=True, score=1.0, reason='The score is 1.00 because the answer is fully relevant, directly addressing the question with no irrelevant statements. Great job staying focused and on-topic!', strict_mode=False, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.013427999999999999, verbose_logs='Statements:\n[\n    "Drug consumption rooms (DCRs) are professionally supervised healthcare facilities where people can consume drugs in safer and more hygienic conditions.",\n    "DCRs are also known as supervised injecting facilities or safer consumption spaces.",\n    "The main aims of DCRs are to reduce health risks such as overdose and infectious diseases.",\n    "DCRs aim to improve public order.",\n    "DCRs aim to connect marginalized drug users with health and social services.",\n    "DCRs reduce the risk of overdose deaths by providing immediate medical assistance.",\

# Output

In [27]:
clean_results = []
for result in all_results:
    if isinstance(result, tuple) and len(result) > 1:
        # Ensure result[1] is a list before iterating
        if isinstance(result[1], list):
            for ind_results in result[1]:
                clean_results.append(ind_results)

print(len(clean_results))


10


In [29]:
pd.DataFrame(clean_results).to_csv("../data/clean_results.csv", index=False)

In [30]:
pd.DataFrame(clean_results)

,name,success,metrics_data,conversational,multimodal,input,actual_output,expected_output,context,retrieval_context,additional_metadata
0,test_case_3,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,What evidence and practices regarding drug con...,Certainly! Here’s a summary of the evidence an...,Nougier and Schatz (2012) provide evidence tha...,None,[Read the Bilateral Schedule announcement. Top...,None
1,test_case_0,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"How are welfare, support services, and housing...",Certainly! Here’s a clear explanation of how *...,"Welfare, support services, and housing focus o...",None,[[cited 2017 Nov 17]. Available from: https://...,None
2,test_case_4,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"How are welfare, support services, and housing...",Certainly! Here’s a clear explanation of how *...,"Welfare, support services, and housing focus o...",None,[Implement a training and orientation program ...,None
3,test_case_1,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Which domains are included under welfare servi...,Certainly! Here’s a breakdown of the **domains...,The domains included are welfare and support s...,None,[Skip to main content\n\nTwitter\n\nFacebook\n...,None
4,test_case_2,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,What evidence and practices regarding drug con...,"Certainly! In their 2012 report, **Nougier and...",Nougier and Schatz (2012) provide evidence tha...,None,[They also worked with mental hospital service...,None
5,test_case_4,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Summarize international policy differences in ...,Certainly! According to Nougier & Schatz (2012...,Nougier & Schatz (2012) highlight that regulat...,None,[What is self-harm? Published 15 September 202...,None
6,test_case_2,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Summarize international policy differences in ...,Nougier & Schatz (2012) highlight significant ...,Nougier & Schatz (2012) highlight that regulat...,None,[Find out more about self-harm. Check out what...,None
7,test_case_1,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,"How are welfare, support services, and housing...",Certainly! Here’s a clear explanation of how *...,"Welfare, support services, and housing focus o...",None,"[17. Nougier, M., and Schatz, E., Drug consump...",None
8,test_case_0,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Which domains are included under welfare servi...,Certainly! Here’s a breakdown of the **domains...,The domains included are welfare and support s...,None,"[From 2021, major reforms are expected across ...",None
9,test_case_3,True,[name='Answer Relevancy' threshold=0.7 success...,False,False,Which domains are included under welfare servi...,Certainly! Here’s a breakdown of the **domains...,The domains included are welfare and support s...,None,"[Welfare and support services, including housi...",None


# Testing